In [0]:
! pip install git+git://github.com/frehseg/nvcc4jupyter.git

  Cloning git://github.com/frehseg/nvcc4jupyter.git to /tmp/pip-req-build-wf6roouo
  Running command git clone -q git://github.com/frehseg/nvcc4jupyter.git /tmp/pip-req-build-wf6roouo
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.1-cp36-none-any.whl size=2095 sha256=f44e00a699d1f7070086a06ab25cb804d0ef77968a3b09d4e4cf0b56afe4e1cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-4zi0a70x/wheels/a4/a5/24/17a2b61f9a725a10155cc6fca753aae28436921df21fa16114
Successfully built NVCCPlugin


In [0]:
%load_ext nvcc_plugin

In [0]:
%%cu
/*
 * Application saxpy avec GPU 
 * y = A.x +  B
 */

#include <stdlib.h>
#include <stdio.h>
#include <cuda.h>

////////////////////////////////////////////////////////////////
//     Initialisation des vecteurs     
////////////////////////////////////////////////////////////////
void init_tab(float *tab, int len, float val) {
    for (int k=0; k<len; k++)   
      tab[k]= val;
}

void affiche_tab(char *chaine, float *tab, int len){
   int k;
   printf("\nLes 10 premiers de %s: \n",chaine);
   for (k=0; k<10; k++) 
      printf("%.2f ",tab[k]);
   printf("\nLes 10 derniers: \n");
   for (k=len-10; k<len; k++) 
      printf("%.2f ",tab[k]);
   printf("\n");
}

__global__ void saxpy(float *tabX, float *tabY, int len, float a){
   int idx = blockIdx.x*blockDim.x + threadIdx.x;
   if (idx < len){
       tabY[idx] = a*tabX[idx] + tabY[idx];
   }
}

int main( int argc, char** argv){ 
    float *tabX_d, *tabX_h;
    float *tabY_d, * tabY_h;
    int len = 100;
    int a = 2;

     /** Initialisation de  nbthreadbyblock et nbblockbygrid  **/
    int nbthreadbyblock = 1024;
    int nbblockbygrid = ceil(len/(float) nbthreadbyblock);

    printf("SAXPY - tableau de %d éléments \n", len);

    /** Allocation memoire sur le host(CPU) **/
    tabX_h=(float *) malloc(sizeof(float) * len);
    init_tab(tabX_h, len , 5.);
    //TODO - allocation de tabY_h
    tabY_h=(float *) malloc(sizeof(float) * len);
    init_tab(tabY_h, len , 5.);


    /** Affichage initial **/
    printf("Affichage initial\n");
    affiche_tab("tabY_h",tabY_h, len);
    affiche_tab("tabX_h",tabX_h, len);


    /** Allocation memoire sur le device(GPU) **/
    cudaMalloc((void**) &tabX_d, sizeof(float) * len);
    // TODO - allocation de tabY_d
    cudaMalloc((void**) &tabY_d, sizeof(float) * len);

    /** Transfert mémoire du host vers le device **/
    cudaMemcpy(tabX_d, tabX_h, len*sizeof(float), cudaMemcpyHostToDevice) ;
    cudaMemcpy(tabY_d, tabY_h, len*sizeof(float), cudaMemcpyHostToDevice) ;

    /** Lancement du kernel **/
    saxpy<<<nbblockbygrid, nbthreadbyblock>>>(tabX_d, tabY_d, len, a);

    /** Transfert mémoire du device vers le host **/
    cudaMemcpy(tabX_h, tabX_d, len*sizeof(float), cudaMemcpyDeviceToHost) ;        cudaMemcpy(tabX_h, tabX_d, len*sizeof(float), cudaMemcpyDeviceToHost) ;
    cudaMemcpy(tabY_h, tabY_d, len*sizeof(float), cudaMemcpyDeviceToHost) ;

    /** Affichage du resultat **/
    printf("Affichage du résultat\n");
    affiche_tab("tabY_h", tabY_h, len);

    /** Libération de la mémoire **/
    cudaFree(tabX_d); cudaFree(tabY_d);
    free(tabX_h); free(tabY_h);  

    printf("Fin du programme\n");
    return EXIT_SUCCESS;
}

SAXPY - tableau de 100 éléments 
Affichage initial

Les 10 premiers de tabY_h: 
5.00 5.00 5.00 5.00 5.00 5.00 5.00 5.00 5.00 5.00 
Les 10 derniers: 
5.00 5.00 5.00 5.00 5.00 5.00 5.00 5.00 5.00 5.00 

Les 10 premiers de tabX_h: 
5.00 5.00 5.00 5.00 5.00 5.00 5.00 5.00 5.00 5.00 
Les 10 derniers: 
5.00 5.00 5.00 5.00 5.00 5.00 5.00 5.00 5.00 5.00 
Affichage du résultat

Les 10 premiers de tabY_h: 
15.00 15.00 15.00 15.00 15.00 15.00 15.00 15.00 15.00 15.00 
Les 10 derniers: 
15.00 15.00 15.00 15.00 15.00 15.00 15.00 15.00 15.00 15.00 
Fin du programme

